In [1]:
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as ts
import pandas as pd
import numpy as np
import sklearn.model_selection as sk
from beakerx import *

df = pd.read_excel('http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/titanic3.xls', 'titanic3', index_col=None, na_values=['NA'], converters={'ticket':str})[['sex','age','name','cabin','ticket','survived']]

table = TableDisplay(df)
table.setStringFormatForTimes(TimeUnit.DAYS)
table.setStringFormatForType(ColumnType.Double, TableDisplayStringFormat.getDecimalFormat(4,6))
table.setStringFormatForColumn("m3", TableDisplayStringFormat.getDecimalFormat(0, 0))
table

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
x = df.loc[:,['survived', 'sex']]
x['age'] = df.loc[:, 'age'].round(-1)

m_mean = x.query('age>=0 & age<70 & sex=="male"').groupby(['age']).mean()
m_std = x.query('age>=0 & age<70 & sex=="male"').groupby(['age']).std() 
m_low = m_mean - m_std
m_hi = m_mean + m_std 

f_mean = x.query('age>=0 & age<70 & sex=="female"').groupby(['age']).mean() 
f_std = x.query('age>=0 & age<70 & sex=="female"').groupby(['age']).std()
f_low = f_mean - f_std
f_hi  = f_mean + f_std

p = Plot()
p.add(Line(y= f_mean.survived, displayName= "Female Survival Rate"))
p.add(Area(y= f_low.values.tolist(), base= f_hi.values.tolist(),
              color= Color(0, 30, 0, 0), displayName= "1 Standard Deviation Female"))
p.add(Line(y= m_mean.survived, displayName= "Male Survival Rate"))
p.add(Area(y= m_low.values.tolist(), base= m_hi.values.tolist(),
              color= Color(255, 0, 0, 50), displayName= "1 Standard Deviation Male"))


In [3]:
# pandas and regex magic to extract the title of the passenger into a 'title' column
df['title'] = df['name'].str.extract('.*, (.*).', expand=False)
x_all = pd.get_dummies(df['title'])
y_all = np.array([ [y==1, y==0] for y in df.survived]).astype(int)

In [4]:
X_train, X_test, y_train, y_test = sk.train_test_split(x_all, y_all, test_size=0.33, random_state=42)

In [5]:
model = Sequential()
model.add(Dense(units=32, input_dim = len(x_all.columns)))
model.add(Dense(units=32, input_dim = len(x_all.columns)))
model.add(Dense(units=2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [6]:
predictions = model.predict(X_test.as_matrix())

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [7]:
# assign the softmax probabilities to a boolean e.g. true or false
predictions_bool = predictions[:,0] > 0.5
# do the same for the response
y_test_bool = y_test[:,0] ==1
accuracy = sum(predictions_bool == y_test_bool) / len(y_test_bool)

In [8]:
print(accuracy)

0.47685185185185186
